## Consider module scoped code to configure deployment environments
The best way to do this is to override parts of your program at startup time to provide different functionality depending on your deployment environment. However, once your deployment environments get complicated, you should consider moving them out of Python constants (like TESTING) and into dedicated configuration files.

## Use repr strings for debugging output
When you're debugging with print, you should repr the value before printing to ensure that any difference in types is clear.

In [1]:
print(repr(5))

5


In [2]:
print(repr('5'))

'5'


For classes we can define our own repr special method:

In [3]:
class BetterClass(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return("BetterClass(%d, %d)" % (self.x, self.y))
    
obj = BetterClass(1, 2)
print(obj)

BetterClass(1, 2)


Or where you don't have control over the class definition:

In [4]:
obj.__dict__

{'x': 1, 'y': 2}

## Test everything with unittest
You should always test your code, regardless of which language it's written in. The unittest module is popular, but Python users appear to be using pytest more these days.

In [5]:
class SomeClass(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def sumXY(self):
        return(self.x + self.y)
    
    def multiplyXY(self):
        return(self.x * self.y)
    

someObj = SomeClass(2, 3)
someObj.sumXY(), someObj.multiplyXY()

(5, 6)

In [10]:
import unittest

class TestSomeClass(unittest.TestCase):
        
    # setUp and tearDown methods run before and after each test respectively
    def setUp(self):
        self.someObj = SomeClass(2, 3)
        print("setUp finished!")
    
    def tearDown(self):
        del self.someObj
        print("tearDown finished!")
        print("")
        
    # the setUpClass and tearDownClass class methods are run when the class is created and destroyed respectively
    @classmethod
    def setUpClass(cls):
        print("setUpClass finished!")
        print("")
    
    @classmethod
    def tearDownClass(cls):
        print("tearDownClass finished!")
        print("")
    
    def test_init(self):
        self.assertEqual(self.someObj.x, 2)
        self.assertEqual(self.someObj.y, 3)
        print("test_init finished!")
    
    def test_sumXY(self):
        self.assertEqual(self.someObj.sumXY(), 5)
        print("test_sumXY finished!")
    
    def test_multiplyXY(self):
        someObj = SomeClass(2, 3)
        self.assertEqual(self.someObj.multiplyXY(), 6)
        print("test_multiplyXY finished!")
    

def main():
    try:
        unittest.main() # this usually works, but it won't in the Jupyter notebook
    except:
        unittest.main(argv=["first-arg-is-ignored"], exit=False) # run this instead

if __name__ == "__main__":
    main()

...

setUpClass finished!

setUp finished!
test_init finished!
tearDown finished!

setUp finished!
test_multiplyXY finished!
tearDown finished!

setUp finished!
test_sumXY finished!
tearDown finished!

tearDownClass finished!




----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


A few things to note:
* We can run tests by inheriting from unittest.TestCase
* Testing methods need to start with the name 'test...'
* There are setUp and tearDown methods that can run before and after each test method. These assist in the actions of one test not polluting another.
* setUpClass and tearDownClass class methods can also be very useful. Creating persistent objects that are time consuming is a good candidate for setUpClass, whereas tearDownClass could do a clean up and delete any files that were created as part of the testing process, for example.
* pytest appears to be more popular that unittest at the time of writing.
* unittest is often used in conjunction with nose and coverage.
* There is no guarantee of the order in which tests will run, but there are ways around this.

In [11]:
unittest.TestCase.__dict__

dict_proxy({'__call__': <function unittest.case.__call__>,
            '__dict__': <attribute '__dict__' of 'TestCase' objects>,
            '__doc__': "A class whose instances are single test cases.\n\n    By default, the test code itself should be placed in a method named\n    'runTest'.\n\n    If the fixture may be used for many test cases, create as\n    many test methods as are needed. When instantiating such a TestCase\n    subclass, specify in the constructor arguments the name of the test method\n    that the instance is to execute.\n\n    Test authors should subclass TestCase for their own tests. Construction\n    and deconstruction of the test's environment ('fixture') can be\n    implemented by overriding the 'setUp' and 'tearDown' methods respectively.\n\n    If it is necessary to override the __init__ method, the base class\n    __init__ method must always be called. It is important that subclasses\n    should not change the signature of their __init__ method, since instan

## Consider interactive debugging with pdb